# DCC-GARCH代码

In [1]:
library(MTS)
library(rmgarch)

载入需要的程辑包：rugarch

载入需要的程辑包：parallel


载入程辑包：‘rugarch’


The following object is masked from ‘package:stats’:

    sigma




In [2]:
# 读取数据
load_txt <- function(file_name){
    data = read.table(file_name, header=T)
    return(data)
    }
    
# 处理数据
# processing <- function(data){
#     return(c(rtn, date))
#     }

# 绘制DCC时变相关系数
plot_corr<-function(date, corr, title){
    plot(date, corr, type='l', main=title, xlab='Year', ylab='Corr')
    }


# 保存DCC时变相关系数
save_txt <- function(corr_df, file_name){
    write.table(corr_df, file=file_name, quote=FALSE)
    }

## MTS的DCC-GARCH

In [3]:
# 计算DCC时变相关系数
dcc_corr <- function(rtn){
    start<-Sys.time()
    m1=dccPre(rtn, include.mean=T,p=0)
    m3=dccFit(m1$sresi,type="Engle")
    corr <- m3$rho.t
    end<-Sys.time()
    running_time<-end-start
    print(running_time)
    return(corr)
    }

## rmgarch的DCC-GARCH

In [3]:
# 计算DCC时变相关系数
dcc_corr2 <- function(data, date){
    start<-Sys.time()
    n=ncol(data)
    p=1;q=1
    meanSpec=list(armaOrder=c(1,0), include.mean=TRUE, archpow=1)
    varSpec=list(method='sGARCH', garchOrder=c(p,q))
    distSpec=c("mvt")
    spec1= ugarchspec(mean.model=meanSpec, variance.model = list(model = "sGARCH", garchOrder = c(1, 1)))
    mySpec = multispec(replicate(n,spec1))
    mySpec = dccspec(mySpec, VAR=F, robust=F, lag=1, lag.max=NULL, lag.criterion=c("AIC"),
                    external.regressors=NULL,
                    robust.control=list(gama=0.25, delta=0.01, nc=10, ns=500),
                    dccOrder=c(1,1), distribution=distSpec, start.pars=list(),
                    fixed.pars=list())
    fit_dcc=dccfit(data=data, mySpec, out.sample=0, solver='solnp', solver.control=list(),
                  fit.control=list(eval.se=TRUE, stationarity=TRUE, scale=FALSE), parallel=TRUE,
                  parallel.control=list(pkg=c("multicore"), cores=2), fit=NULL, VAR.fit=NULL)
    a = rcor(fit_dcc)
    con_corr = matrix(a, ncol=length(date))
    con_corr = t(con_corr)
    end<-Sys.time()
    running_time<-end-start
    print(running_time)
    return(con_corr)
    }

## 对比测试

In [5]:
da = read.table("datasets/ch7/m-ibmspko-6111.txt", header=T)
date <- as.Date(as.character(da$date, format='%Y%m%d'), '%Y%m%d')
rtn = log(da[,2:4]+1)

In [58]:
con_corr=dcc_corr(rtn)
#par(mfrow=c(3,1))
#plot(date, con_corr[,4], type='l', main='IBM-SP', xlab='Year', ylab='Corr')
#plot(date, con_corr[,7],  type='l', main='IBM-KO',xlab='Year', ylab='Corr')
#plot(date, con_corr[,8], type='l', main='SP-KO', xlab='Year', ylab='Corr')

Sample mean of the returns:  0.00772774 0.005023909 0.01059521 
Component:  1 
Estimates:  0.000419 0.126738 0.788309 
se.coef  :  0.000162 0.035404 0.055645 
t-value  :  2.593451 3.579734 14.16673 
Component:  2 
Estimates:  9e-05 0.127725 0.836053 
se.coef  :  4.1e-05 0.03084 0.031723 
t-value  :  2.20126 4.141593 26.35484 
Component:  3 
Estimates:  0.000256 0.098706 0.830358 
se.coef  :  8.5e-05 0.022361 0.03344 
t-value  :  3.015323 4.414111 24.83091 
Estimates:  0.9126534 0.04531516 8.624328 
st.errors:  0.02947897 0.0127403 1.332697 
t-values:   30.95947 3.556835 6.471336 
Time difference of 6.829443 secs


In [6]:
con_corr=dcc_corr2(rtn, date)
# par(mfrow=c(3,1))
# plot(date, con_corr[,4], type='l', main='IBM-SP', xlab='Year', ylab='Corr')
# plot(date, con_corr[,7],  type='l', main='IBM-KO',xlab='Year', ylab='Corr')
# plot(date, con_corr[,8], type='l', main='SP-KO', xlab='Year', ylab='Corr')

Time difference of 2.536855 secs
